In [18]:
import requests
from lxml import html
import pandas as pd
import time
import base64
import random

In [43]:
def prePage(url):
    time.sleep(2)
    page =  requests.get(url)
    tree = html.fromstring(page.content)
    
    return tree

In [44]:
def metroList():
    data = pd.DataFrame()
    page = prePage('https://zoon.ru/msk/beauty/metro/')
    metro = []
    links = []
    count = []
    c = 0
    
    lol = {}
    
    for one in page.cssselect('li.feed-box'):
        #print(one.cssselect('a.block-link.title')[0].text_content())
        #print(one.cssselect('a.block-link.title')[0].attrib.get('href'))
        #print(one.cssselect('div.description a')[0].text_content())
        #metro.append(one.cssselect('a.block-link.title')[0].text_content())
        lol[one.cssselect('a.block-link.title')[0].attrib.get('href')] = one.cssselect('a.block-link.title')[0].text_content()
        #links.append(one.cssselect('a.block-link.title')[0].attrib.get('href'))
        #s = one.cssselect('div.description a')[0].text_content()
        #s = int(s[:s.find(' ')])
        #count.append(s)
        #c += s
        
    #data['metro'] = metro
    #data['url'] = links
    #data['count'] = count
    
    #print(c)
    #res = [data, 
    #return links
    return lol

In [45]:
def parse(urls):
    log = []
    for url in urls.keys():
        page =  prePage(url)
        #tree = html.fromstring(page.content)
        s = str(page.cssselect('script.points')[0].text_content())+"#h"+urls[url]+"#"
        #log.append(page.cssselect('script.points')[0].text_content())
        log.append(s)
        '''
        head = tree.head.getchildren()
        for item in head:
            if(item.tag == 'title'):
                print('Title:', item.text)
            elif(item.tag == 'meta'):
                if(item.attrib.get('name') == 'description'):
                    print('Description:',item.attrib.get('content'))
                if(item.attrib.get('name') == 'keywords'):
                    print('Keywords:',item.attrib.get('content'))
        
        print(tree.cssselect('script.points')[0].text_content())'''
        
    return log
        
       
        

In [46]:
z = metroList()

In [47]:
%time log = parse(z)

CPU times: user 14.2 s, sys: 416 ms, total: 14.6 s
Wall time: 10min 18s


In [48]:
log[0][log[0].rfind('#h')+2:len(log[0])-1]

'Южная'

# Unicalize

In [49]:
import json

In [50]:
def to_json(data):
    res = []
    for d in data:
        metro = d[d.rfind('#h')+2:len(d)-1]
        nd = json.loads(d[:d.rfind('#h')])
        for i in nd:
            i['metro'] = metro
            res.append(i)
    
    return res

In [51]:
json_log = to_json(log)

In [166]:
def to_csv(data):
    res = pd.DataFrame()
    
    res['ID'] = [i['id'] for i in data]
    res['title'] = [i['name'].replace('&amp;', '&').replace('&#039;', '').replace('&quot;', '"') for i in data]
    res['metro'] = [i['metro'] for i in data]
    res['types'] = [i['type'] for i in data]
    res['premium'] = [i['premium'] for i in data]
    res['lon'] = [i['lonlat'][0] for i in data]
    res['lat'] = [i['lonlat'][1] for i in data]
    
    res = res.drop_duplicates()
    
    return res

In [167]:
log = to_csv(json_log)

In [168]:
log.premium.unique()

array([ True, False], dtype=bool)

In [169]:
LEG = log.copy()

In [170]:
del LEG['metro']

In [171]:
LEG = LEG.drop_duplicates()

In [172]:
%time LEG['metro_cat'] = LEG['ID'].apply(lambda x: ','.join(log[log.ID == x].metro.tolist())).astype(str)

CPU times: user 44.4 s, sys: 156 ms, total: 44.6 s
Wall time: 44.5 s


In [173]:
LEG.premium.unique()

array([ True, False], dtype=bool)

In [174]:
LEG.premium.value_counts()

False    10418
True       463
Name: premium, dtype: int64

In [175]:
LEG[LEG.ID == '52045a91a0f302db0f00001f']

,ID,title,types,premium,lon,lat,metro_cat
1896,52045a91a0f302db0f00001f,Салон красоты 10 & 10 в Басманном,beauty,False,37.637201,55.762474,"Сретенский бульвар,Тургеневская,Чистые пруды,К..."


In [176]:
log[log.ID == '52c0249a40c0886b7c8bf72a']

,ID,title,metro,types,premium,lon,lat
8752,52c0249a40c0886b7c8bf72a,Салон красоты АРТ Студия (ШАРМ) на Моршанской ...,Лермонтовский проспект,beauty,False,37.856604,55.696149


In [177]:
LEG.to_csv('ZoonBeaty - all_salon - v1.csv', index=False, encoding = 'utf-8')

# Reviews,Prices and URLs

In [150]:
l = 'https://zoon.ru/js.php?area=service&action=CommentList&organization=51945fcca0f302a235000042&sort=default'

Url парсить нет необходимости, ZoonID ведет на карточку товара. Следовательно, парсим отзывы и отдельно карточку. <br>
2 типа карточек: <b>free & paid</b><br>
Телефон парсится через href без 'tel:+' <br>
*Можно встроить JSON парсинг в основной парсинг по метро*<br>
3 базы - все салоны, данные из карточек, отзывы<br>
TypesFilter

# URLs - NotPremium

In [226]:
def createProxy():
    proxy = {}
    data = pd.read_csv('hidemy_proxy_http.csv')
    ip_list = list(data.IP)
    port_list = list(data.Port)
    type_list = list(data.Type)
    for i in range(0, len(ip_list)):
        proxy[i+1] = type_list[i].lower()+';'+str(ip_list[i])+':'+str(port_list[i])
    print('Proxy:',len(proxy))
    return proxy

In [232]:
def prePageV2(url, proxy):
    rnd = random.randint(1, len(proxy)+1)
    proxies = {proxy[rnd][:proxy[rnd].find(';')]: proxy[rnd][proxy[rnd].find(';')+1:]
                #'https':'185.8.5.219:443'
            }
    time.sleep(3)
    page = requests.post(url, proxies=proxies)
    tree = html.fromstring(page.content)
    #print(page.status_code == 200)
    return tree

In [233]:
def not_premium(data):
    data = data.copy()
    metro = {}
    tel = {}
    rating = {}
    ratingCount = {}
    reviewCount = {}
    address = {}
    timing = {}
    desc = {}
    sites = {}
    
    proxy = createProxy()
    
    k = 1
    for ID in list(data.ID):
        try:
            page = prePageV2('https://zoon.ru/msk/beauty/'+str(ID), proxy)
        except:
            page = prePageV2('https://zoon.ru/msk/beauty/'+str(ID), proxy)
        '''
        try:
            #m = page.cssselect('div.service-page-header h1')[0].text_content().replace('\n','').replace('\t','')
            #print(k)
            #print(m)
        except:
            page = prePageV2('https://zoon.ru/msk/beauty/'+str(ID), proxy)
            break
            print('FAIL')
            return 'FAIL'''
        
        try:
            metro[ID] = page.cssselect('div.map-text span a')[0].text_content()
        except:
            metro[ID] = '-'
            
        try:
            tel[ID] = page.cssselect('a.tel-phone.js-phone-number')[0].attrib.get('href')[5:]
        except:
            tel[ID] = '-'
            
        try:
            rating[ID] = page.xpath('//span[@itemprop="ratingValue"]/text()')[0]
        except:
            rating[ID] = 0
            
        try:
            reviewCount[ID] = page.xpath('//span[@itemprop="reviewCount"]/text()')[0]
        except:
            reviewCount[ID] = 0
        
        try:
            ratingCount[ID] = page.xpath('//span[@itemprop="ratingCount"]/text()')[0]
        except:
            ratingCount[ID] = 0
        
        try:
            address[ID] = page.cssselect('address.iblock')[0].text_content()
        except:
            address[ID] = '-'
        
        try:
            s = page.cssselect('dl.fluid.uit-cover dd.simple-text div')[0].text_content()
            timing[ID] = ' '.join(s.split(' ')[2:])
        except:
            timing[ID] = '-'
            
        try:
            s1 =  page.cssselect('dd.simple-text.js-desc.oh.word-break')[0].text_content()
            desc[ID] = s1.replace('...','').replace('показать еще','').replace('\t',' ').replace('\n',' ')
        except:
            desc[ID] = '-'
        
        try:
            sites[ID] = page.cssselect('div.service-website a')[0].text_content()
        except:
            sites[ID] = '-'
            
        #s2 = page.cssselect('div.service-description-block.invisible-links div.params-list')
        #s3 = s2[s2.find('data = "')+8:s2.find('";')]
        #print(s2)
        #print(s3)
        print('Success ID:', ID, 'Left: ',len(list(data.ID))-k)
        k+=1
    
    data['metro'] = data.ID.apply(lambda x: metro[x]).astype(str)
    data['tel'] = data.ID.apply(lambda x: tel[x]).astype(str)
    data['rating'] = data.ID.apply(lambda x: rating[x]).astype(float)
    data['ratingCount'] = data.ID.apply(lambda x: ratingCount[x]).astype(int)
    data['reviewCount'] = data.ID.apply(lambda x: reviewCount[x]).astype(int)
    data['address'] = data.ID.apply(lambda x: address[x]).astype(str)
    data['timing'] = data.ID.apply(lambda x: timing[x]).astype(str)
    data['description'] = data.ID.apply(lambda x: desc[x]).astype(str)
    data['site'] = data.ID.apply(lambda x: sites[x]).astype(str)
    
    #data[''] = data.ID.apply(lambda x: )
    
    #one_photo - parsing from home_listings
    #price_list - separately parser - ?
    #roubrics:tags - trouble JS - parsing from home_listings - OK
    return data

In [234]:
#data = pd.read_csv('ZoonBeaty - all_salon - v1.csv')
data = LEG

In [237]:
LOG = not_premium(data[:1])

Proxy: 75
Success ID: 56207b20aecee0c62f8b4568 Left:  0


In [238]:
LOG

,ID,title,types,premium,lon,lat,metro_cat,metro,tel,rating,ratingCount,reviewCount,address,timing,description,site
0,56207b20aecee0c62f8b4568,Студия BIGODINI STUDIO на Варшавском шоссе,beauty,True,37.616032,55.629097,Южная,Южная,74951146300,4.5,96,36,"Москва, Варшавское шоссе, 122, 1 этаж",ежедневно: 10:00 - 21:00График работы в праздн...,Добро пожаловать в студию красоты «BIGOD...,bigodini-studio.ru


In [259]:
LOG.to_csv('ZoonBeauty - 200-360.csv', index=False, encoding='utf-8')

In [ ]:
p = 